In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import cv2

In [2]:
def convolve(image, K):
    # grab the spatial dimensions of the image and kernel
    (iH, iW) = image.shape[:2]
    (kH, kW) = K.shape[:2]
    
    pad = (kW - 1) //2
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    output = np.zeros((iH, iW), dtype="float")
    
    for y in np.arange(pad, iH +  pad):
        for x in np.arange(pad, iW + pad):
            # extract ROI of the image by extracting the center region of the current (x, y) coordinates dimension
            roi = image[y - pad:y + pad + 1, x - pad:x + pad +1]

            # perform the convolution by taking the element-wise multiplication btn ROI and the kernel, then summing the matrix
            k = (roi * K).sum()

            # store the convolved value in the output (x, y)-coordinate of the output image
            output[y - pad, x - pad] = k
            
    # rescale the output image to be in range [0, 255]
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")
    
    return output

In [3]:
# Construct average blurring kernels or filters used to smooth an image
smallBlur = np.ones((7, 7), dtype='float') * (1.0 / (7 * 7))
largeBlur = np.ones((21, 21), dtype='float') * (1.0 / (21 * 21))

print("[INFO] small blur filter")
print(smallBlur)

print()
print("[INFO] large blur filter")
print(largeBlur)

[INFO] small blur filter
[[0.02040816 0.02040816 0.02040816 0.02040816 0.02040816 0.02040816
  0.02040816]
 [0.02040816 0.02040816 0.02040816 0.02040816 0.02040816 0.02040816
  0.02040816]
 [0.02040816 0.02040816 0.02040816 0.02040816 0.02040816 0.02040816
  0.02040816]
 [0.02040816 0.02040816 0.02040816 0.02040816 0.02040816 0.02040816
  0.02040816]
 [0.02040816 0.02040816 0.02040816 0.02040816 0.02040816 0.02040816
  0.02040816]
 [0.02040816 0.02040816 0.02040816 0.02040816 0.02040816 0.02040816
  0.02040816]
 [0.02040816 0.02040816 0.02040816 0.02040816 0.02040816 0.02040816
  0.02040816]]

[INFO] large blur filter
[[0.00226757 0.00226757 0.00226757 0.00226757 0.00226757 0.00226757
  0.00226757 0.00226757 0.00226757 0.00226757 0.00226757 0.00226757
  0.00226757 0.00226757 0.00226757 0.00226757 0.00226757 0.00226757
  0.00226757 0.00226757 0.00226757]
 [0.00226757 0.00226757 0.00226757 0.00226757 0.00226757 0.00226757
  0.00226757 0.00226757 0.00226757 0.00226757 0.00226757 0.0022675

In [4]:
# Construct sharpen and laplacian filters or kernels
sharpen = np.array((
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]), dtype='int')

laplacian = np.array((
    [0, 1, 0],
    [1, -4, 1],
    [0, 1, 0]), dtype='int')

# Construct an edge-like region detector along x, y
sobelX = np.array((
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]), dtype='int')

sobelY = np.array((
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]), dtype='int')

# Construct an emboss kernel/filter
emboss = np.array((
    [-2, -1, 0],
    [-1, 1, 1],
    [0, 1, 2]), dtype='int')

In [5]:
# Construct a kernelBank
kernelBank = (
    ('small_blur', smallBlur),
    ('large_blur', largeBlur),
    ('sharpen', sharpen),
    ('laplacian', laplacian),
    ('sobel_x', sobelX),
    ('sobel_y', sobelY),
    ('emboss', emboss))

In [6]:
# Load input image
image_url = 'C:/Users/Tajr/Desktop/iss.jpg'
image = cv2.imread(image_url)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# cv2.imshow('Original', gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
# Loop over the kernel bank and apply kernels to an input image
for (kernelName, K) in kernelBank:
    # Apply the kernel to grayscale image using our convolve function and openCV's filter2D function
    print('[INFO] applying {} kernel'. format(kernelName))
    convolvedOutput = convolve(gray, K)
    openCVoutput = cv2.filter2D(gray, -1, K)
    
    # Printing the output
    cv2.imshow('Original', gray)
    cv2.imshow('{} - convolve'.format(kernelName), convolvedOutput)
    cv2.imshow('{} - opencv'.format(kernelName), openCVoutput)
    cv2.waitKey(1)
#     cv2.destroyAllWindows()

[INFO] applying small_blur kernel
[INFO] applying large_blur kernel
[INFO] applying sharpen kernel
[INFO] applying laplacian kernel
[INFO] applying sobel_x kernel
[INFO] applying sobel_y kernel
[INFO] applying emboss kernel
